<a href="https://colab.research.google.com/github/Mreeb/YOLO-V8-CLASSIFICATION-ON_Dermnet/blob/master/1_Classification_YOLOv8_6_Classes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Connect Runtime To a GPU

In [2]:
!nvidia-smi

Mon Jan  8 18:29:28 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

# Installing Libraies


In [2]:
!pip install --upgrade numpy
!pip install ultralytics==8.0.58 roboflow numpy==1.24.2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 40.3 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.23.5
    Uninstalling numpy-1.23.5:
      Successfully uninstalled numpy-1.23.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
lida 0.0.10 requires fastapi, which is not installed.
lida 0.0.10 requires kaleido, which is not installed.
lida 0.0.10 requires python-multipart, which is not installed.
lida 0.0.10 requires uvicorn, which is not installed.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.8/486.8 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.9/69.9 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 31.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 254.1/254.1 kB 25.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

# Mounting Drive

In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="C4pHQj9DsJmYjHwOLRr3")
project = rf.workspace("care-pftlv").project("skin-disease-classification-6o4dw")
dataset = project.version(4).download("folder")

loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to Skin-Disease-Classification-4 in folder:: 100%|██████████| 14187/14187 [00:05<00:00, 2580.12it/s]


In [ ]:
from ultralytics import YOLO

model_path = 'yolov8x-cls.pt'  # @param {type:"string"}
data_path = '/content/Skin-Disease-Classification-4'  # @param {type:"string"}

epochs = 5  # @param {type:"integer"}
img_size = 640  # @param {type:"integer"}
amp = False # @param {type:"boolean"}
cache = True # @param {type:"boolean"}
resume = True # @param {type:"boolean"}

patience = 10 # @param {type:"integer"}
# Create YOLO model and train
model = YOLO(model_path)
model.train(data=data_path, epochs=epochs, imgsz=img_size, amp = amp, cache = cache, patience=patience)


In [ ]:
!kill 8858

In [ ]:
%reload_ext tensorboard
%tensorboard --logdir runs/classify/train2

In [ ]:
%reload_ext tensorboard

In [ ]:
path = "/content/runs/classify/train2/weights/best.pt"
from google.colab import files
files.download(path)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [2]:
from ultralytics import YOLO
import torch
model = YOLO('best.pt')

In [5]:
results = model("/content/Skin-Disease-Classification-4/test/Acne/07Acne081101_jpg.rf.ccddcff5717ca510a40c7da7ad765067.jpg", save = True)


image 1/1 /content/Skin-Disease-Classification-4/test/Acne/07Acne081101_jpg.rf.ccddcff5717ca510a40c7da7ad765067.jpg: 224x224 Acne 0.97, Atopic-Dermatitis 0.01, Basal-cell-carcinoma 0.01, Unlabeled 0.00, Fissured-Skin 0.00, 21.8ms
Speed: 0.4ms preprocess, 21.8ms inference, 0.1ms postprocess per image at shape (1, 3, 224, 224)
Results saved to runs/classify/predict


In [6]:
if isinstance(results, list):
    # If the results are a list, then get the first result
    results = results[0]
classes = {"Acne":0, "Atopic-Dermatits":1, "Basal-cell-carcinoma" : 2 , "Fissured-Skin" : 3, "Melanoma" : 4, "Nail Onychopathies" : 5}


max_value, max_index = torch.max(results.probs, dim=0)
print(f" Disease : {list(classes.keys())[int(max_index)]}\n Confidence : {round(float(max_value), 2)}")


 Disease : Acne
 Confidence : 0.97


# Validation Accuracy

In [3]:
metrics = model.val()  # no arguments needed, dataset and settings remembered
metrics.top1   # top1 accuracy
metrics.top5   # top5 accuracy

Ultralytics YOLOv8.0.58 🚀 Python-3.10.12 torch-2.1.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
YOLOv8x-cls summary (fused): 133 layers, 56132167 parameters, 0 gradients, 153.8 GFLOPs
               classes   top1_acc   top5_acc: 100%|██████████| 16/16 [00:01<00:00, 10.95it/s]
                   all      0.735      0.988
Speed: 0.1ms preprocess, 5.0ms inference, 0.0ms loss, 0.0ms postprocess per image
Results saved to runs/classify/val2


0.9877550601959229

In [5]:
import os

Actual = list()
Predicted = list()

folder_path = "/content/Skin-Disease-Classification-4/test"
exclude_folder = "Unlabeled"

folders = [f for f in os.listdir(folder_path) if os.path.isdir(os.path.join(folder_path, f)) and f != exclude_folder]
classes = {"Acne": 0, "Atopic-Dermatitis": 1, "Basal-cell-carcinoma": 2, "Fissured-Skin": 3, "Melanoma": 4, "Nail Onychopathies": 5}

for folder in folders:
    folder_full_path = os.path.join(folder_path, folder)
    images = [img for img in os.listdir(folder_full_path) if img.endswith(('.jpg', '.jpeg', '.png', '.gif', '.bmp'))]

    print(f"Images in {folder}:")
    for image in images:
        results = model(os.path.join(folder_full_path, image))
        if isinstance(results, list):
            results = results[0]

        max_value, max_index = torch.max(results.probs, dim=0)
        val = list(classes.keys())[int(max_index)]
        Predicted.append(classes.get(val))
        Actual.append(classes.get(folder))
        max_value, max_index = torch.max(results.probs, dim=0)
        #print(f"Disease: {list(classes.keys())[int(max_index)]} === Actual: {folder}")

    #print()



image 1/1 /content/Skin-Disease-Classification-4/test/Atopic-Dermatitis/05atopic031306_jpg.rf.48787719b5cc7ecd9f0c007004da40d3.jpg: 224x224 Acne 0.31, Basal-cell-carcinoma 0.29, Melanoma 0.19, Atopic-Dermatitis 0.12, Fissured-Skin 0.09, 43.8ms
Speed: 0.4ms preprocess, 43.8ms inference, 0.1ms postprocess per image at shape (1, 3, 224, 224)

image 1/1 /content/Skin-Disease-Classification-4/test/Atopic-Dermatitis/rhus-dermatitis-22_jpg.rf.b77ff87ddc6d5887ecd8f9006107a6c0.jpg: 224x224 Atopic-Dermatitis 0.33, Basal-cell-carcinoma 0.25, Acne 0.21, Fissured-Skin 0.13, Melanoma 0.05, 41.7ms
Speed: 0.3ms preprocess, 41.7ms inference, 0.1ms postprocess per image at shape (1, 3, 224, 224)



Images in Atopic-Dermatitis:


image 1/1 /content/Skin-Disease-Classification-4/test/Atopic-Dermatitis/seborrheic-dermatitis-11_jpg.rf.86bcc169df1e2d733f22e06159d37b8a.jpg: 224x224 Acne 0.49, Atopic-Dermatitis 0.26, Fissured-Skin 0.11, Basal-cell-carcinoma 0.09, Melanoma 0.04, 24.1ms
Speed: 0.3ms preprocess, 24.1ms inference, 0.1ms postprocess per image at shape (1, 3, 224, 224)

image 1/1 /content/Skin-Disease-Classification-4/test/Atopic-Dermatitis/05AtopicFossa2q_jpg.rf.e8d3fab5453f5f01e0740d31b8bf370b.jpg: 224x224 Atopic-Dermatitis 0.71, Fissured-Skin 0.28, Unlabeled 0.01, Melanoma 0.00, Nail Onychopathies 0.00, 17.7ms
Speed: 0.3ms preprocess, 17.7ms inference, 0.1ms postprocess per image at shape (1, 3, 224, 224)

image 1/1 /content/Skin-Disease-Classification-4/test/Atopic-Dermatitis/05AtopicFace2_jpg.rf.c5e0ee34c0757b44aff439228546078c.jpg: 224x224 Acne 0.78, Atopic-Dermatitis 0.19, Fissured-Skin 0.02, Unlabeled 0.01, Basal-cell-carcinoma 0.00, 24.9ms
Speed: 0.3ms preprocess, 24.9ms inference, 0.1ms postproce

Images in Acne:


image 1/1 /content/Skin-Disease-Classification-4/test/Acne/4453-13248-20010-22040tn_jpg.rf.9f534dde5a8a4a1e2eff90f64e3acf40.jpg: 224x224 Melanoma 0.32, Basal-cell-carcinoma 0.32, Acne 0.27, Atopic-Dermatitis 0.06, Fissured-Skin 0.03, 11.2ms
Speed: 0.3ms preprocess, 11.2ms inference, 0.1ms postprocess per image at shape (1, 3, 224, 224)

image 1/1 /content/Skin-Disease-Classification-4/test/Acne/acne-cystic-134_jpg.rf.cc4e732195c4a8e773234f703dc09af7.jpg: 224x224 Acne 0.98, Atopic-Dermatitis 0.01, Fissured-Skin 0.00, Unlabeled 0.00, Basal-cell-carcinoma 0.00, 11.0ms
Speed: 0.3ms preprocess, 11.0ms inference, 0.0ms postprocess per image at shape (1, 3, 224, 224)

image 1/1 /content/Skin-Disease-Classification-4/test/Acne/acne-cystic-109_jpg.rf.1ac4e0124cf6c6c883f5b58c4624a940.jpg: 224x224 Atopic-Dermatitis 0.46, Fissured-Skin 0.35, Melanoma 0.08, Acne 0.06, Nail Onychopathies 0.02, 10.9ms
Speed: 0.3ms preprocess, 10.9ms inference, 0.1ms postprocess per image at shape (1, 3, 224, 224)

im

...

image 1/1 /content/Skin-Disease-Classification-4/test/Melanoma/blue-nevus-ota-9_jpg.rf.2b3cbc08d1334c4e016c8a6108829b35.jpg: 224x224 Atopic-Dermatitis 0.41, Fissured-Skin 0.35, Acne 0.14, Melanoma 0.03, Unlabeled 0.03, 16.5ms
Speed: 0.3ms preprocess, 16.5ms inference, 0.1ms postprocess per image at shape (1, 3, 224, 224)

image 1/1 /content/Skin-Disease-Classification-4/test/Melanoma/ISIC_6653225_jpg.rf.ade8a3a11b8f58dc4e895af99b0ac397.jpg: 224x224 Basal-cell-carcinoma 0.61, Melanoma 0.15, Atopic-Dermatitis 0.14, Acne 0.06, Fissured-Skin 0.02, 10.1ms
Speed: 0.3ms preprocess, 10.1ms inference, 0.1ms postprocess per image at shape (1, 3, 224, 224)

image 1/1 /content/Skin-Disease-Classification-4/test/Melanoma/nevus-spilus-26_jpg.rf.b3826dbc79d2fd78fc4007d96376dcc7.jpg: 224x224 Melanoma 0.83, Basal-cell-carcinoma 0.13, Atopic-Dermatitis 0.02, Acne 0.01, Fissured-Skin 0.01, 10.4ms
Speed: 0.3ms preprocess, 10.4ms inference, 0.0ms postprocess per image at shape (1, 3, 224, 224)

image 1/1 /

Images in Fissured-Skin:


image 1/1 /content/Skin-Disease-Classification-4/test/Fissured-Skin/prurigo-nodularis-33_jpg.rf.e39927eed5cfce829322770a28c12597.jpg: 224x224 Fissured-Skin 0.60, Atopic-Dermatitis 0.36, Acne 0.01, Unlabeled 0.01, Melanoma 0.01, 10.2ms
Speed: 0.5ms preprocess, 10.2ms inference, 0.0ms postprocess per image at shape (1, 3, 224, 224)

image 1/1 /content/Skin-Disease-Classification-4/test/Fissured-Skin/chapped-fissured-feet-26_jpg.rf.18dbfbb70b9911ccd10b0440990f421b.jpg: 224x224 Fissured-Skin 0.50, Atopic-Dermatitis 0.41, Nail Onychopathies 0.04, Unlabeled 0.02, Acne 0.02, 10.2ms
Speed: 0.2ms preprocess, 10.2ms inference, 0.0ms postprocess per image at shape (1, 3, 224, 224)

image 1/1 /content/Skin-Disease-Classification-4/test/Fissured-Skin/Dyshidrosis-4_jpg.rf.ac7b8bde2973e6238b17009d36ce7d54.jpg: 224x224 Atopic-Dermatitis 0.46, Fissured-Skin 0.36, Acne 0.06, Nail Onychopathies 0.04, Basal-cell-carcinoma 0.03, 10.2ms
Speed: 0.2ms preprocess, 10.2ms inference, 0.1ms postprocess per image 

Images in Nail Onychopathies:


image 1/1 /content/Skin-Disease-Classification-4/test/Nail Onychopathies/pincer-nails-17_jpg.rf.7721813dbe11e42d8299b0b02e7c5d78.jpg: 224x224 Nail Onychopathies 0.85, Fissured-Skin 0.07, Atopic-Dermatitis 0.06, Acne 0.01, Melanoma 0.01, 9.4ms
Speed: 0.2ms preprocess, 9.4ms inference, 0.0ms postprocess per image at shape (1, 3, 224, 224)

image 1/1 /content/Skin-Disease-Classification-4/test/Nail Onychopathies/distal-subungual-onychomycosis-148_jpg.rf.72ffb9df6d1df8d8b22cd36852b63985.jpg: 224x224 Nail Onychopathies 0.96, Atopic-Dermatitis 0.02, Fissured-Skin 0.01, Melanoma 0.01, Basal-cell-carcinoma 0.01, 9.4ms
Speed: 0.3ms preprocess, 9.4ms inference, 0.1ms postprocess per image at shape (1, 3, 224, 224)

image 1/1 /content/Skin-Disease-Classification-4/test/Nail Onychopathies/mucous-cyst-40_jpg.rf.c38176623695972dce0222436f4b2cea.jpg: 224x224 Nail Onychopathies 0.97, Fissured-Skin 0.02, Atopic-Dermatitis 0.01, Melanoma 0.00, Acne 0.00, 9.3ms
Speed: 0.3ms preprocess, 9.3ms inference, 0

# Acuracy On TestSet

In [6]:
from sklearn.metrics import accuracy_score

# Calculate accuracy
accuracy = accuracy_score(Actual, Predicted)
print(f"Accuracy: {accuracy * 100:.2f}%")

Accuracy: 74.07%


In [7]:
from sklearn.metrics import confusion_matrix

confusion_mat = confusion_matrix(Actual, Predicted)
print("Confusion Matrix:")
print(confusion_mat)

Confusion Matrix:
[[49  4  2  2  1  0]
 [11 10  5  6  2  0]
 [ 1  2 12  0  0  1]
 [ 1  6  1  9  0  1]
 [ 1  3  5  0 24  0]
 [ 0  3  1  4  0 76]]


In [8]:
from sklearn.metrics import precision_score, recall_score, f1_score

precision = precision_score(Actual, Predicted, average='weighted')
recall = recall_score(Actual, Predicted, average='weighted')
f1 = f1_score(Actual, Predicted, average='weighted')

print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1-Score: {f1:.2f}")


Precision: 0.76
Recall: 0.74
F1-Score: 0.74


In [9]:
from sklearn.metrics import classification_report

report = classification_report(Actual, Predicted)
print("Classification Report:")
print(report)


Classification Report:
              precision    recall  f1-score   support

           0       0.78      0.84      0.81        58
           1       0.36      0.29      0.32        34
           2       0.46      0.75      0.57        16
           3       0.43      0.50      0.46        18
           4       0.89      0.73      0.80        33
           5       0.97      0.90      0.94        84

    accuracy                           0.74       243
   macro avg       0.65      0.67      0.65       243
weighted avg       0.76      0.74      0.74       243

